# 财务比率分析

## 📋 概述

财务比率分析是基本面分析中最重要的工具之一，通过计算和分析各种财务比率，可以全面评估企业的财务状况、盈利能力、偿债能力和运营效率。本章节将详细介绍各类财务比率的计算方法、分析要点和实际应用，包括盈利能力比率、偿债能力比率、运营效率比率和成长性比率等。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际数据计算和可视化财务比率，深入理解财务分析。


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib seaborn akshare

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import akshare as ak
from datetime import datetime, timedelta

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("whitegrid")

print("环境准备完成！")


## 📖 核心内容

### 1. 财务比率分类

财务比率通常分为以下几大类：

1. **盈利能力比率**：评估企业盈利能力的指标
2. **偿债能力比率**：评估企业偿债能力的指标
3. **运营效率比率**：评估企业运营效率的指标
4. **成长性比率**：评估企业成长能力的指标

让我们先创建一个财务比率计算的工具类：


In [ ]:
# 财务比率计算类
class FinancialRatioAnalyzer:
    """财务比率分析器"""
    
    def __init__(self, balance_sheet, income_statement, cash_flow):
        """
        初始化财务比率分析器
        
        参数:
        balance_sheet: 资产负债表数据（DataFrame）
        income_statement: 利润表数据（DataFrame）
        cash_flow: 现金流量表数据（DataFrame）
        """
        self.bs = balance_sheet
        self.is_df = income_statement
        self.cf = cash_flow
    
    def profitability_ratios(self):
        """计算盈利能力比率"""
        ratios = {}
        
        # 毛利率 = (营业收入 - 营业成本) / 营业收入
        if '营业收入' in self.is_df.columns and '营业成本' in self.is_df.columns:
            ratios['毛利率'] = (self.is_df['营业收入'] - self.is_df['营业成本']) / self.is_df['营业收入'] * 100
        
        # 净利率 = 净利润 / 营业收入
        if '净利润' in self.is_df.columns and '营业收入' in self.is_df.columns:
            ratios['净利率'] = self.is_df['净利润'] / self.is_df['营业收入'] * 100
        
        # ROE = 净利润 / 平均净资产
        if '净利润' in self.is_df.columns and '所有者权益合计' in self.bs.columns:
            ratios['ROE'] = self.is_df['净利润'] / self.bs['所有者权益合计'] * 100
        
        # ROA = 净利润 / 平均总资产
        if '净利润' in self.is_df.columns and '资产总计' in self.bs.columns:
            ratios['ROA'] = self.is_df['净利润'] / self.bs['资产总计'] * 100
        
        return ratios
    
    def solvency_ratios(self):
        """计算偿债能力比率"""
        ratios = {}
        
        # 流动比率 = 流动资产 / 流动负债
        if '流动资产合计' in self.bs.columns and '流动负债合计' in self.bs.columns:
            ratios['流动比率'] = self.bs['流动资产合计'] / self.bs['流动负债合计']
        
        # 速动比率 = (流动资产 - 存货) / 流动负债
        if '流动资产合计' in self.bs.columns and '存货' in self.bs.columns and '流动负债合计' in self.bs.columns:
            ratios['速动比率'] = (self.bs['流动资产合计'] - self.bs['存货']) / self.bs['流动负债合计']
        
        # 资产负债率 = 负债总额 / 资产总额
        if '负债合计' in self.bs.columns and '资产总计' in self.bs.columns:
            ratios['资产负债率'] = self.bs['负债合计'] / self.bs['资产总计'] * 100
        
        # 利息保障倍数 = (利润总额 + 利息费用) / 利息费用
        # 简化计算，假设利息费用在财务费用中
        if '利润总额' in self.is_df.columns and '财务费用' in self.is_df.columns:
            interest_expense = abs(self.is_df['财务费用'])
            ratios['利息保障倍数'] = (self.is_df['利润总额'] + interest_expense) / (interest_expense + 1)
        
        return ratios
    
    def efficiency_ratios(self):
        """计算运营效率比率"""
        ratios = {}
        
        # 应收账款周转率 = 营业收入 / 平均应收账款
        if '营业收入' in self.is_df.columns and '应收账款' in self.bs.columns:
            ratios['应收账款周转率'] = self.is_df['营业收入'] / (self.bs['应收账款'] + 1)
        
        # 存货周转率 = 营业成本 / 平均存货
        if '营业成本' in self.is_df.columns and '存货' in self.bs.columns:
            ratios['存货周转率'] = self.is_df['营业成本'] / (self.bs['存货'] + 1)
        
        # 总资产周转率 = 营业收入 / 平均总资产
        if '营业收入' in self.is_df.columns and '资产总计' in self.bs.columns:
            ratios['总资产周转率'] = self.is_df['营业收入'] / self.bs['资产总计']
        
        # 固定资产周转率 = 营业收入 / 平均固定资产
        if '营业收入' in self.is_df.columns and '固定资产合计' in self.bs.columns:
            ratios['固定资产周转率'] = self.is_df['营业收入'] / (self.bs['固定资产合计'] + 1)
        
        return ratios
    
    def growth_ratios(self, previous_period_data):
        """计算成长性比率（需要上一期数据）"""
        ratios = {}
        
        if previous_period_data is None:
            return ratios
        
        prev_is = previous_period_data['income_statement']
        prev_bs = previous_period_data['balance_sheet']
        
        # 营业收入增长率
        if '营业收入' in self.is_df.columns and '营业收入' in prev_is.columns:
            ratios['营业收入增长率'] = (self.is_df['营业收入'] - prev_is['营业收入']) / prev_is['营业收入'] * 100
        
        # 净利润增长率
        if '净利润' in self.is_df.columns and '净利润' in prev_is.columns:
            ratios['净利润增长率'] = (self.is_df['净利润'] - prev_is['净利润']) / prev_is['净利润'] * 100
        
        # 总资产增长率
        if '资产总计' in self.bs.columns and '资产总计' in prev_bs.columns:
            ratios['总资产增长率'] = (self.bs['资产总计'] - prev_bs['资产总计']) / prev_bs['资产总计'] * 100
        
        return ratios
    
    def cash_flow_ratios(self):
        """计算现金流比率"""
        ratios = {}
        
        # 经营活动现金流/净利润
        if '经营活动产生的现金流量净额' in self.cf.columns and '净利润' in self.is_df.columns:
            ratios['现金流/净利润'] = self.cf['经营活动产生的现金流量净额'] / (self.is_df['净利润'] + 1)
        
        # 经营活动现金流/营业收入
        if '经营活动产生的现金流量净额' in self.cf.columns and '营业收入' in self.is_df.columns:
            ratios['现金流/营业收入'] = self.cf['经营活动产生的现金流量净额'] / (self.is_df['营业收入'] + 1)
        
        return ratios

print("财务比率分析器类定义完成！")


### 2. 盈利能力比率

盈利能力比率反映企业赚取利润的能力，是评估企业价值的重要指标。

#### 2.1 主要指标

**毛利率（Gross Margin）**：
- 公式：毛利率 = (营业收入 - 营业成本) / 营业收入 × 100%
- 含义：反映产品的盈利能力
- 标准：通常认为30%以上较好，但行业差异很大

**净利率（Net Margin）**：
- 公式：净利率 = 净利润 / 营业收入 × 100%
- 含义：反映企业的整体盈利能力
- 标准：通常认为10%以上较好

**ROE（净资产收益率）**：
- 公式：ROE = 净利润 / 平均净资产 × 100%
- 含义：反映股东权益的盈利能力
- 标准：通常认为15%以上较好

**ROA（总资产收益率）**：
- 公式：ROA = 净利润 / 平均总资产 × 100%
- 含义：反映总资产的盈利能力
- 标准：通常认为5%以上较好

让我们用示例数据演示：


In [ ]:
# 创建示例财务数据
# 注意：实际应用中应该从公司年报中获取真实数据

# 示例资产负债表数据
balance_sheet = pd.DataFrame({
    '资产总计': [1000000, 1100000, 1200000],
    '流动资产合计': [400000, 450000, 500000],
    '存货': [100000, 120000, 140000],
    '应收账款': [80000, 90000, 100000],
    '固定资产合计': [500000, 550000, 600000],
    '负债合计': [400000, 450000, 500000],
    '流动负债合计': [200000, 220000, 240000],
    '所有者权益合计': [600000, 650000, 700000]
}, index=['2022', '2023', '2024'])

# 示例利润表数据
income_statement = pd.DataFrame({
    '营业收入': [500000, 600000, 700000],
    '营业成本': [300000, 360000, 420000],
    '净利润': [50000, 60000, 70000],
    '利润总额': [60000, 72000, 84000],
    '财务费用': [5000, 6000, 7000]
}, index=['2022', '2023', '2024'])

# 示例现金流量表数据
cash_flow = pd.DataFrame({
    '经营活动产生的现金流量净额': [60000, 70000, 80000]
}, index=['2022', '2023', '2024'])

# 创建分析器
analyzer = FinancialRatioAnalyzer(balance_sheet, income_statement, cash_flow)

# 计算盈利能力比率
profitability = analyzer.profitability_ratios()

# 可视化盈利能力比率
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 毛利率
if '毛利率' in profitability:
    axes[0, 0].bar(profitability['毛利率'].index, profitability['毛利率'].values, color='skyblue')
    axes[0, 0].axhline(y=30, color='red', linestyle='--', label='参考线(30%)')
    axes[0, 0].set_title('毛利率 (%)', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('百分比')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

# 净利率
if '净利率' in profitability:
    axes[0, 1].bar(profitability['净利率'].index, profitability['净利率'].values, color='lightgreen')
    axes[0, 1].axhline(y=10, color='red', linestyle='--', label='参考线(10%)')
    axes[0, 1].set_title('净利率 (%)', fontsize=14, fontweight='bold')
    axes[0, 1].set_ylabel('百分比')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

# ROE
if 'ROE' in profitability:
    axes[1, 0].bar(profitability['ROE'].index, profitability['ROE'].values, color='coral')
    axes[1, 0].axhline(y=15, color='red', linestyle='--', label='参考线(15%)')
    axes[1, 0].set_title('ROE (净资产收益率) (%)', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('百分比')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

# ROA
if 'ROA' in profitability:
    axes[1, 1].bar(profitability['ROA'].index, profitability['ROA'].values, color='gold')
    axes[1, 1].axhline(y=5, color='red', linestyle='--', label='参考线(5%)')
    axes[1, 1].set_title('ROA (总资产收益率) (%)', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('百分比')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印比率值
print("盈利能力比率：")
for ratio_name, ratio_values in profitability.items():
    print(f"\n{ratio_name}:")
    for period, value in ratio_values.items():
        print(f"  {period}: {value:.2f}%")


### 3. 偿债能力比率

偿债能力比率反映企业偿还债务的能力，是评估企业财务风险的重要指标。

#### 3.1 主要指标

**流动比率（Current Ratio）**：
- 公式：流动比率 = 流动资产 / 流动负债
- 含义：反映企业短期偿债能力
- 标准：通常认为2:1较为理想

**速动比率（Quick Ratio）**：
- 公式：速动比率 = (流动资产 - 存货) / 流动负债
- 含义：更严格地反映短期偿债能力
- 标准：通常认为1:1较为理想

**资产负债率（Debt-to-Asset Ratio）**：
- 公式：资产负债率 = 负债总额 / 资产总额 × 100%
- 含义：反映企业财务杠杆水平
- 标准：通常认为50%以下较为安全

**利息保障倍数（Interest Coverage Ratio）**：
- 公式：利息保障倍数 = (利润总额 + 利息费用) / 利息费用
- 含义：反映企业支付利息的能力
- 标准：通常认为3倍以上较好

让我们计算和可视化偿债能力比率：


In [ ]:
# 计算偿债能力比率
solvency = analyzer.solvency_ratios()

# 可视化偿债能力比率
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 流动比率
if '流动比率' in solvency:
    axes[0, 0].bar(solvency['流动比率'].index, solvency['流动比率'].values, color='skyblue')
    axes[0, 0].axhline(y=2, color='red', linestyle='--', label='参考线(2.0)')
    axes[0, 0].set_title('流动比率', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('比率')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

# 速动比率
if '速动比率' in solvency:
    axes[0, 1].bar(solvency['速动比率'].index, solvency['速动比率'].values, color='lightgreen')
    axes[0, 1].axhline(y=1, color='red', linestyle='--', label='参考线(1.0)')
    axes[0, 1].set_title('速动比率', fontsize=14, fontweight='bold')
    axes[0, 1].set_ylabel('比率')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

# 资产负债率
if '资产负债率' in solvency:
    axes[1, 0].bar(solvency['资产负债率'].index, solvency['资产负债率'].values, color='coral')
    axes[1, 0].axhline(y=50, color='red', linestyle='--', label='参考线(50%)')
    axes[1, 0].set_title('资产负债率 (%)', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('百分比')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

# 利息保障倍数
if '利息保障倍数' in solvency:
    axes[1, 1].bar(solvency['利息保障倍数'].index, solvency['利息保障倍数'].values, color='gold')
    axes[1, 1].axhline(y=3, color='red', linestyle='--', label='参考线(3.0)')
    axes[1, 1].set_title('利息保障倍数', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('倍数')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印比率值
print("偿债能力比率：")
for ratio_name, ratio_values in solvency.items():
    print(f"\n{ratio_name}:")
    for period, value in ratio_values.items():
        if '率' in ratio_name and '%' not in str(value):
            print(f"  {period}: {value:.2f}")
        else:
            print(f"  {period}: {value:.2f}")


### 4. 运营效率比率

运营效率比率反映企业资产和资源的利用效率，是评估企业管理水平的重要指标。

#### 4.1 主要指标

**应收账款周转率（Receivables Turnover）**：
- 公式：应收账款周转率 = 营业收入 / 平均应收账款
- 含义：反映应收账款回收速度
- 标准：越高越好

**存货周转率（Inventory Turnover）**：
- 公式：存货周转率 = 营业成本 / 平均存货
- 含义：反映存货销售速度
- 标准：越高越好

**总资产周转率（Total Asset Turnover）**：
- 公式：总资产周转率 = 营业收入 / 平均总资产
- 含义：反映总资产利用效率
- 标准：越高越好

**固定资产周转率（Fixed Asset Turnover）**：
- 公式：固定资产周转率 = 营业收入 / 平均固定资产
- 含义：反映固定资产利用效率
- 标准：越高越好

让我们计算和可视化运营效率比率：


In [ ]:
# 计算运营效率比率
efficiency = analyzer.efficiency_ratios()

# 可视化运营效率比率
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 应收账款周转率
if '应收账款周转率' in efficiency:
    axes[0, 0].plot(efficiency['应收账款周转率'].index, efficiency['应收账款周转率'].values, 
                    marker='o', linewidth=2, markersize=8, color='skyblue')
    axes[0, 0].set_title('应收账款周转率', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('次数')
    axes[0, 0].grid(True, alpha=0.3)

# 存货周转率
if '存货周转率' in efficiency:
    axes[0, 1].plot(efficiency['存货周转率'].index, efficiency['存货周转率'].values, 
                    marker='o', linewidth=2, markersize=8, color='lightgreen')
    axes[0, 1].set_title('存货周转率', fontsize=14, fontweight='bold')
    axes[0, 1].set_ylabel('次数')
    axes[0, 1].grid(True, alpha=0.3)

# 总资产周转率
if '总资产周转率' in efficiency:
    axes[1, 0].plot(efficiency['总资产周转率'].index, efficiency['总资产周转率'].values, 
                    marker='o', linewidth=2, markersize=8, color='coral')
    axes[1, 0].set_title('总资产周转率', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('次数')
    axes[1, 0].grid(True, alpha=0.3)

# 固定资产周转率
if '固定资产周转率' in efficiency:
    axes[1, 1].plot(efficiency['固定资产周转率'].index, efficiency['固定资产周转率'].values, 
                    marker='o', linewidth=2, markersize=8, color='gold')
    axes[1, 1].set_title('固定资产周转率', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('次数')
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印比率值
print("运营效率比率：")
for ratio_name, ratio_values in efficiency.items():
    print(f"\n{ratio_name}:")
    for period, value in ratio_values.items():
        print(f"  {period}: {value:.2f}")


### 5. 现金流比率

现金流比率反映企业利润的现金含量，是评估企业盈利质量的重要指标。

#### 5.1 主要指标

**现金流/净利润**：
- 公式：现金流/净利润 = 经营活动现金流净额 / 净利润
- 含义：反映利润的现金含量
- 标准：通常认为大于1较好

**现金流/营业收入**：
- 公式：现金流/营业收入 = 经营活动现金流净额 / 营业收入
- 含义：反映收入的现金回收率
- 标准：通常认为应该大于0

让我们计算和可视化现金流比率：


In [ ]:
# 计算现金流比率
cash_flow_ratios = analyzer.cash_flow_ratios()

# 可视化现金流比率
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 现金流/净利润
if '现金流/净利润' in cash_flow_ratios:
    axes[0].bar(cash_flow_ratios['现金流/净利润'].index, 
                cash_flow_ratios['现金流/净利润'].values, color='skyblue')
    axes[0].axhline(y=1, color='red', linestyle='--', label='参考线(1.0)')
    axes[0].set_title('现金流/净利润', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('比率')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

# 现金流/营业收入
if '现金流/营业收入' in cash_flow_ratios:
    axes[1].bar(cash_flow_ratios['现金流/营业收入'].index, 
                cash_flow_ratios['现金流/营业收入'].values, color='lightgreen')
    axes[1].axhline(y=0, color='red', linestyle='--', label='参考线(0)')
    axes[1].set_title('现金流/营业收入', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('比率')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印比率值
print("现金流比率：")
for ratio_name, ratio_values in cash_flow_ratios.items():
    print(f"\n{ratio_name}:")
    for period, value in ratio_values.items():
        print(f"  {period}: {value:.2f}")


## 💡 关键要点总结

1. **财务比率分类**：盈利能力、偿债能力、运营效率、现金流比率
2. **分析方法**：横向比较、纵向比较、行业比较
3. **关键指标**：ROE、ROA、流动比率、资产负债率
4. **注意事项**：不能孤立地看单个指标，要结合行业特点分析

## 🛠️ 工具与软件

### 数据获取

- **公司官网**：投资者关系页面
- **交易所网站**：上交所、深交所、港交所
- **财经网站**：东方财富、同花顺、雪球
- **数据平台**：Wind、Choice、Tushare、AKShare

### 分析工具

- **Excel**：计算比率、制作图表
- **Python**：批量分析、数据可视化（本Notebook）
- **专业软件**：财务分析软件

### 注意事项

- ✅ **数据准确性**：使用官方发布的财务报表
- ✅ **时效性**：关注最新报表，也要看历史趋势
- ✅ **行业差异**：不同行业的财务比率差异很大
- ⚠️ **综合分析**：不能孤立地看单个指标，要综合分析
- ⚠️ **报表质量**：注意审计意见，关注报表附注

## 🔗 相关知识点

- [资产负债表分析](./资产负债表分析.md)
- [利润表分析](./利润表分析.md)
- [现金流量表分析](./现金流量表分析.md)
- [绝对估值法](../02_公司估值/绝对估值法.md)
- [相对估值法](../02_公司估值/相对估值法.md)

## 📚 拓展阅读

- 《财务报表分析》- 财务报表分析经典教材
- 《公司财务原理》- 深入理解财务比率
- 各公司年报、季报 - 实际案例分析
